<a href="https://colab.research.google.com/github/Tchy258/Proyectos-discretas/blob/main/Proyecto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysat
!pip install folium
!pip install python-sat[pblib,aiger]
!pip install python-sat
!pip install -U python-sat

In [ ]:
import random
import numpy as np
from pysat.solvers import Minisat22
import math

#Primero hay que ver cuantas n clausulas hay que recibir (cuantos array hay que recibir)

cantidadClausulas=int(input('Cantidad de clausulas: '))

#Luego hay que ver cuantas proposiciones hay

cantidadProposiciones=int(input('Cantidad de proposiciones: '))

#Hay que generar aleatoriedad

#negadorAleatorio recibe una proposicion x, y dependiendo del valor de la variable negar, la niega o la mantiene
def negadorAleatorio(x):
    negar=random.random() #negar es un numero aleatorio entre 0 y 1
    if negar>=0.5: #Si negar es mayor que 0.5
        return x #La proposición se mantiene
    else: #Si no
        return -x #Se niega

#generarListaAleatoria recibe la cantidad de clausulas y la cantidad de proposiciones y genera una lista al azar de formulas en 3-CNF
def generarListaAleatoria(clausulas,proposiciones):
    listaGenerada=np.zeros(clausulas,3) #Primero se crea una matriz de nx3 donde n es la cantidad de clausulas (o conjunciones)
    for i in range(0,clausulas): #Por cada clausula
        for j in range(0,3): #Para cada proposición dentro de una clausula
            proposicion=negadorAleatorio(random.randint(0,proposiciones)) #Elegir un literal al azar y al mismo tiempo negarlo aleatoriamente
            listaGenerada[i][j]=proposicion #Agregar el literal aleatorizado a la clausula i en el espacio j de la disyunción
    return listaGenerada


#valuacionObjetivo devuelve la combinacion de valores de verdad donde
#cada valor por si solo satisface la disyunción
def valuacionObjetivo(literales):
  objetivo=np.full(3,-1) #objetivo es un array que determina el objetivo de valores de verdad segun el signo de los literales
  for i in range (0,literales):
    if literales[i]>0:
      objetivo[i]=1 #1 si es que el literal no esta negado
    else:
      objetivo[i]=0 #0 si es que está negado
  return objetivo

#sigmaGorro devuelve el valor de verdad de una disyunción de literales
#donde basta que uno corresponda
def sigmaGorro(valores,objetivo):
  almenosUna=False
  for i in range (0,valores):
    if valores[i]==objetivo[i]:
      almenosUna=True #Si el valor i de la fila de la tabla de verdad corresponde al objetivo, se continúa evaluando y se recuerda que almenos 1 proposición si corresponde
  if almenosUna: #Si había al menos 1 proposición
    return True #Esta combinación de valores sirve
  else: #Si no
    return False #No sirve


#buscarValuacionDesde busca una valuacion de tabla de verdad a partir de una cierta fila, devuelve el indice de la primera verdadera, o -1 si es que no hay
def buscarValuacionDesde(indice,valuaciones):
  if indice<8: #Si no se han recorrido todas las filas de la subtabla de verdad
    for i in range(indice,8): #Buscar a partir de la fila indice
      if valuaciones[i]==True: #Si se encuentra una fila que cumple
        return i #Devolver la fila
  return -1 #Si no hay ninguna que cumpla o indice es mayor o igual que 8, retornar -1

#asignarValoresDeVerdad coloca el valor correspondiente según dice la tabla cada proposición de la clausula a revisar y luego retorna el arreglo
#de valores junto con un valor de True o False donde True significa que no hay contradicciones y False significa que si
def asignarValoresDeVerdad(filaDeTabla,clausula,valoresDeVerdad):
  #Se convierten los valores de cada literal de la clausula a valor absoluto y se les resta 1, para utilizarlos de indice
  #Así el indice 0 del arreglo valoresDeVerdad indica cual debe ser el valor del literal 1, el indice 1 el del 2, etc
  literal1=int(abs(clausula[0])) - 1
  literal2=int(abs(clausula[1])) - 1
  literal3=int(abs(clausula[2])) - 1
  #Se crea una copia del arreglo valoresDeVerdad en caso de que se encuentren contradicciones y no perder el original
  nuevosValores=np.copy(valoresDeVerdad)
  #Cada uno de los ifs anidados siguientes son los distintos casos de las 8 filas de la tabla de verdad
  if filaDeTabla==0: #Caso 1,1,1
    if valoresDeVerdad[literal1]==-1: #Si el literal no había sido revisado antes
      nuevosValores[literal1]=1 #Asignarle un 1
    elif valoresDeVerdad[literal1]==0: #Si ya tenía un 0 (contradicción)
      return (valoresDeVerdad,False) 
      #Retornar los valores que ya se tenían y un False que indica que los valores actuales no satisfacen la conjunción
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==1: #Caso 1,1,0
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=0
    elif valoresDeVerdad[literal3]==1: #Si el literal ya tenía un 1 (contradicción)
      return (valoresDeVerdad,False)
  if filaDeTabla==2: #Caso 1,0,1
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==3: #Caso 1,0,0
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  if filaDeTabla==4: #Caso 0,1,1
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==5: #Caso 0,1,0
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  if filaDeTabla==6: #Caso 0,0,1
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==7: #Caso 0,0,0
    if valoresDeVerdad[literal1]==-1:
      nuevosValores[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      nuevosValores[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      nuevosValores[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  return (nuevosValores,True)
  



#evaluarLista recibe una lista de literales en 3-CNF y dice si es satisfactible o no y si es que si, retorna una combinación que la satisface.
def evaluarLista(lista,clausulas,proposiciones):
  #subTabla es una tabla de verdad que va a ser utilizada por cada clausula 3-CNF
  subTabla=np.array([1,1,1],[1,1,0],[1,0,1],[1,0,0],[0,1,1],[0,1,0],[0,0,1],[0,0,0])
  combinacionesSatisfactorias=np.full((clausulas,8),False) #Registro de cuales combinaciones satisfacen la disyunción de literales por cada clausula
  for i in range(0,clausulas): #Por cada conjunción
    objetivo=valuacionObjetivo(lista[i]) #Ver la valuación objetivo
    for j in range(0,8): #Por cada combinación de valores de la tabla de verdad
      satisface=sigmaGorro(subTabla[j],lista[i]) #Ver si corresponde
      if satisface: #Si corresponde
        combinacionesSatisfactorias[i][j]=True #Registrarla
  caminosViables=np.full((clausulas,8),(-1,-1)) #Registro de caminos posibles, -1 es desconocido, 1 es viable y 0 es inviable, el segundo numero indica el origen del camino
  #-1 es el valor por defecto y si el camino es viable, este valor cambia al numero de la fila de la tabla de verdad de la clausula anterior
  inicio=0 #Indice para elegir el inicio del camino (la combinacion de la subtabla de verdad de la primera combinación que no ha sido revisada)
  while inicio < 8: #Mientras no haya revisado todos los inicios de camino posibles
    valoresDeVerdad=np.full(proposiciones,-1) #Registro de valores de verdad para cada literal para evaluar cada combinación de combinaciones posible y encontrar una satisfactoria
    indiceSubtabla1=buscarValuacionDesde(inicio,combinacionesSatisfactorias[0]) #Elegir una combinación de la subtabla de verdad de la clausula inicial a partir del indice de inicio
    for i in range(0,clausulas): #Por cada clausula
      if i==0: #Si estoy en la primera clausula
        disyuncion=combinacionesSatisfactorias[i][indiceSubtabla1] #Usar el inicio del camino
        (valoresDeVerdad,satisfactible)=asignarValoresDeVerdad(disyuncion,lista[i],valoresDeVerdad) #Actualizar valores de verdad iniciales
      else: #Si no estoy en la primera
        for j in range(0,8): #Por cada fila j de la subtabla de verdad de la clausula i
          indiceCamino=buscarValuacionDesde(j,combinacionesSatisfactorias[i]) #Buscar una combinación satisfactoria desde j en adelante
          disyuncion=combinacionesSatisfactorias[i][indiceCamino] #Elegirla
          (valoresDeVerdad,satisfactible)=asignarValoresDeVerdad(disyuncion,lista[i],valoresDeVerdad) #Chequear si no hay contradicciones con los valores ya asignados
          if satisfactible: #Si no hay contradicciones
            break #Dejar de buscar entre las j filas de la clausula i, ya que ya se encontró una combinación de valores de verdad viable hasta el momento y pasar a la clausula i+1
      if not satisfactible: #Si se terminó de buscar por las j filas
        break #Dejar de recorrer las i clausulas, ya que no hay combinaciones de valores de verdad para esta fila inicial de la primera clausula
    if satisfactible: #Si se terminaron de recorrer todas las clausulas y se encontró un camino que las satisface
      return (valoresDeVerdad,True) #Retornar los valores de verdad para cada literal y un True para corroborar que esta combinación sirve
    else: #Si no
      inicio+=1 #Volver al siguiente punto de partida
  return (None,False) #Si ya se analizaron todas las combinaciones posibles, devolver None y un False que indica que esta lista de literales es insatisfactible
        
        
